In [1]:
import numpy as np
from scipy.integrate import odeint
# Libraries for the Metabolic block
import os
from os.path import join
data_dir="."
import matplotlib.pyplot as plt
import cobra
from cobra.io import read_sbml_model, write_sbml_model
model = cobra.io.read_sbml_model("iMM904.xml")


solution=model.optimize()
model.summary()

Metabolite,Reaction,Flux,C-Number,C-Flux
glc__D_e,EX_glc__D_e,10,6,100.00%
nh4_e,EX_nh4_e,1.611,0,0.00%
o2_e,EX_o2_e,2,0,0.00%
pi_e,EX_pi_e,0.05691,0,0.00%
so4_e,EX_so4_e,0.02225,0,0.00%
Metabolite,Reaction,Flux,C-Number,C-Flux
co2_e,EX_co2_e,-18.02,1,36.29%
etoh_e,EX_etoh_e,-15.82,2,63.70%
for_e,EX_for_e,-0.001488,1,0.00%
h2o_e,EX_h2o_e,-5.64,0,0.00%


In [9]:
solution_fba = model.optimize()
print(solution_fba)

sumaFlujos = 0
for reaction in model.reactions:
    sumaFlujos += solution_fba[reaction.id]**2
print(model.objective.value, sumaFlujos)


model.reactions.BIOMASS_SC5_notrace.bounds=(solution_fba['BIOMASS_SC5_notrace'],solution_fba['BIOMASS_SC5_notrace'])

for i, reaction in enumerate(model.reactions):
    if i == 0:
        expresion_euclideana = model.reactions.get_by_id(reaction.id).flux_expression**2
    else:
        expresion_euclideana += model.reactions.get_by_id(reaction.id).flux_expression**2

objetivo_euclideano = model.problem.Objective(expresion_euclideana,direction="min")
model.objective = objetivo_euclideano
#print(objetivo_euclideano)
#Nueva distribución de flujo

solucion_minflux = model.optimize()

sumaFlujos = 0 
for reaction in model.reactions:
    sumaFlujos += solucion_minflux[reaction.id]**2
print(solucion_minflux['BIOMASS_SC5_notrace'], model.objective.value, sumaFlujos)

<Solution 0.488 at 0x7fd430664f50>
0.4881232305900678 12791.737357180187
0.4881232305900678 11918.834673707976 11918.83467370784


In [13]:

# Definición del bloque Cinético

def bloqueCinetico(Glu,Eth,Gly,Cit,Lac):
    # Parámetros
    vGmax, K_Glu, K_Eth = 22.5, 0.88, 6.74
    f_Eth,f_Gly, f_Cit, f_Lac = 0.112, 0.273, 0.169, 0.137
    v_Glu=(vGmax*Glu/(K_Glu+Glu))*(1/(1+Eth/K_Eth))
    LB_Eth = -v_Glu*f_Eth
    LB_Gly = -v_Glu*f_Gly
    LB_Cit = -v_Glu*f_Cit
    LB_Lac = -v_Glu*f_Lac
    return [v_Glu,LB_Eth,LB_Gly,LB_Cit,LB_Lac]


#Definición del bloque Metabólico

def bloqueMetabolico(v_Glu,LB_Eth,LB_Gly,LB_Cit,LB_Lac):
    model.reactions.get_by_id("EX_glc__D_e").bounds      = (-v_Glu,-v_Glu) # Define el flujo de glucosa
    model.reactions.get_by_id("EX_etoh_e").lower_bound   = LB_Eth
    model.reactions.get_by_id("EX_glyc_e").lower_bound   = LB_Gly
    model.reactions.get_by_id("EX_cit_e").lower_bound    = LB_Cit
    model.reactions.get_by_id("EX_lac__D_e").lower_bound = LB_Lac
    u = solution.objective_value
    v_Eth = model.reactions.get_by_id("EX_etoh_e").x
    v_Gly = model.reactions.get_by_id("EX_glyc_e").x
    v_Cit = model.reactions.get_by_id("EX_cit_e").x
    v_Lac = model.reactions.get_by_id("EX_lac__D_e").x
    return [u, v_Eth, v_Gly, v_Cit, v_Lac]


# Definición del Bloque Dinámico

def f(y,t,parametros):
    V,VX,VGlu,VEth,VGly,VCit,VLac = y #Current Values
    F, u, v_Glu, v_Eth, v_Gly, v_Cit, v_Lac = parametros #Unpack parameters
    # Tal que F=Flowrate(A,B,C,S,Y,V,X,t)
    Glu_F = 300
    MW_Glu,MW_Eth,MW_Gly,MW_Cit,MW_Lac = [0.18,.046,.092,.192,.090]
    
    derivadas = [F,                        #dV/dt
                 u*VX,                     #dVX/dt
                 F*Glu_F-v_Glu*MW_Glu*(VX),#VGlu/dt
                 v_Eth*MW_Eth*(VX),        #dVEth/dt
                 v_Gly*MW_Gly*(VX),        #dVGly/dt
                 v_Cit*MW_Cit*(VX),        #dVCit/dt
                 v_Lac*MW_Lac*(VX)]        #dVLac/dt
    return derivadas


def bloqueDinamico(y,parametros,ti,tf):
    time = np.linspace(ti,tf,200)
    #F, u, v_Glu, v_Eth, v_Gly, v_Cit, v_Lac = parametros
    solcn = odeint(f,y,time,args=(parametros))
    #Obtención de la solución en el último tiempo (tf)
    V = solcn[-1,0]
    X,Glu,Eth,Gly,Cit,Lac=soln[-1,1:7]/V
    return [V,X,Glu,Eth,Gly,Cit,Lac]


# Guardado de resultados durante la trayectoria de la fermentación
u_path,V_path,X_path = [],[],[]
Glu_path,Eth_path,Gly_path,Cit_path,Lac_path = [],[],[],[],[]
v_Glu_path=[]

def savepath(u,V,X,Glu,Eth,Gly,Cit,Lac,v_Glu):
    global u_path,V_path,X_path # Se declaran variables globales
    global Glu_path,Eth_path,Gly_path,Cit_path,Lac_path
    global v_Glu_path
    u_path += [u]
    V_path += [V]
    X_path += [X]
    Glu_path += [Glu]
    Eth_path += [Eth]
    Gly_path += [Gly]
    Cit_path += [Cit]
    Lac_path += [Lac]
    v_Glu_path += [v_Glu]

In [14]:
### Implementación del Ciclo Iterativo
##
# Condiciones Iniciales
Glu = 30
Eth,Gly,Cit,Lac = [3,0.75,0.1,0.2]
V,X = [0.5,1]
F = 0.25

#Definición de la simulación en el tiempo
time = np.linspace(0,20,200)

for i in range(len(time)):#El ciclo recorre el tiempo definido(20h,200x)
    # 1ro Para el bloque cinético:
    # Dadas las condiciones de glucosa y etanol,
    # Se calculan los flujos de glucosa y los límites superiores e inferiores de los demás metabolitos
    v_Glu,LB_Eth,LB_Gly,LB_Cit,LB_Lac = bloqueCinetico(Glu,Eth,Gly,Cit,Lac)
    # 2do Para el bloque metabólico:
    # Dados los valores de v_Glu,LB_Eth,LB_Gly,LB_Cit,LB_Lac
    # Se calcula la tasa de crecimiento de biomasa (u) y los flujos metabólicos de los metabolitos
    u, v_Eth, v_Gly, v_Cit, v_Lac = bloqueMetabolico(v_Glu,LB_Eth,LB_Gly,LB_Cit,LB_Lac)
    # 3ro Para el bloque dinámico:
    # Dados los valores de u, V, los flujos de (v_Eth, v_Gly, v_Cit, v_Lac)
    # y las concentraciones de (X,Glu,Eth,Gly,Cit,Lac)
    # Actualizamos el volumen de la reacción V, X y Glu,Eth,Gly,Cit,Lac
    if i == len(time)-1: continue 
    y = [V,X*V,Glu*V,Eth*V,Gly*V,Cit*V,Lac*V]
    parametros = [F(time[i]),u,v_Glu, v_Eth, v_Gly, v_Cit, v_Lac]
    V,X,Glu,Eth,Gly,Cit,Lac = bloqueDinamico(y, parametros, time[i],time[i+1])
    # Guardamos los resultados de las trayectorias a lo largo de la fermentación
    savepath(u,V,X,Glu,Eth,Gly,Cit,Lac,v_Glu)

OptimizationError: Likely no solution exists. Original solver message: CPLEX Error  1217: No solution exists..